In [ ]:
# Imports
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
from sklearn.pipeline import make_pipeline

Task 1: Exploratory data analytics on text data


#### a)

In [ ]:
data_X = pd.read_csv("train.csv")
data_y = pd.read_csv("test.csv")
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(data_X["Text"])

tfidDF = vectorizer.transform(data_X["Text"])

#number of articles and features
num_articles = tfidDF.shape[0]
num_features = tfidDF.shape[1]

print(f"Number of articles: {num_articles}")
print(f"Number of extracted features: {num_features}")

#5 example articles
fivearticles = data_X["Text"].head(5)
features = tfidDF[:5].toarray()
feature_names = vectorizer.get_feature_names_out()

example_df = pd.DataFrame(features, columns=feature_names, index=fivearticles)

print(example_df.head())

For these articles, we peformed vectorisation using inverse document frequency as a way of tokenising the texts. From this, we observed 13518 unique words becoming our features from 428 articles.

#### b)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

word_matrix = vectorizer.fit_transform(data_X["Text"])
word_hz = word_matrix.sum(axis=0).A1
terms = vectorizer.get_feature_names_out()

term_freq_df = pd.DataFrame({'term': terms, 'frequency': word_hz})
term_freq_df = term_freq_df.sort_values(by='frequency', ascending=False)

#plot 1
plt.figure(figsize=(15, 8))
top50 = term_freq_df.head(50)
plt.bar(top50['term'], top50['frequency'])
plt.xticks(rotation=90)
plt.xlabel('Terms')
plt.ylabel('Frequency')
plt.title('Top-50 Term Frequency Distribution Across Entire Dataset')
plt.show()

categories = data_X['Category'].unique()
term_hz = {}

for category in categories:
    category_texts = data_X[data_X['Category'] == category]['Text']
    category_word_matrix = vectorizer.transform(category_texts)
    category_word_hz = category_word_matrix.sum(axis=0).A1
    term_hz[category] = category_word_hz

#plot 2&3
fig, axes = plt.subplots(len(categories), 1, figsize=(15, 8 * len(categories)))

for i, category in enumerate(categories):
    term_freqDF = pd.DataFrame({'term': terms, 'frequency': term_hz[category]})
    term_freqDF = term_freqDF.sort_values(by='frequency', ascending=False)
    top50_terms = term_freqDF.head(50)
    
    axes[i].bar(top50_terms['term'], top50_terms['frequency'])
    axes[i].set_xticks(range(len(top50_terms['term'])))
    axes[i].set_xticklabels(top50_terms['term'], rotation=90)
    axes[i].set_xlabel('Terms')
    axes[i].set_ylabel('Frequency')
    axes[i].set_title(f'Top-50 Term Frequency Distribution for Category: {category}')

plt.tight_layout()
plt.show()

#plot 4
plt.figure(figsize=(10, 6))
category_distribution = data_X['Category'].value_counts()
category_distribution.plot(kind='bar')
plt.xlabel('Category')
plt.ylabel('Number of Articles')
plt.title('Category Distribution')
plt.show()

From looking at the above plots, we notice "said' was mentioned the most frequently across all articles, surpassing the closest frequent word by over two times. The remaining top term frequencies seems quite consistent ranging from 600 down to 200. We also see "said" is most frequent in the category tech compared to entertainment. Overall, the data seems fairly balanced with similar number of articles for each category. 

Task 2: Perform classification models (NB, kNN, SVM, NNs) to build article classifiers using the given dataset.

## Naive Bayes

In [ ]:
# soft SVM
y = data_X["Category"]

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

X_train, X_valid, y_train, y_valid = train_test_split(word_matrix, y, test_size = 0.2, random_state = 42)

# use cross validation to choose C
c_param = {'C': [0.01, 0.1, 1, 10, 100]}
grid_search = GridSearchCV(SVC(kernel = "linear"), c_param, cv = 5)
grid_search.fit(X_train, y_train)
best_c = grid_search.best_params_['C']
print(f'Best C parameter: {best_c}')

# train final model with best c
linear_svm = SVC(kernel = "linear", C = best_c)
linear_svm.fit(X_train, y_train)

y_pred_linear = linear_svm.predict(X_valid)
print('Soft-Margin Linear SVM:')
print('Accuracy:', accuracy_score(y_valid, y_pred_linear))
print('Classification Report:')
print(classification_report(y_valid, y_pred_linear))

## kNN

## SVM

In [ ]:
# soft SVM
y = data_X["Category"]

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

X_train, X_valid, y_train, y_valid = train_test_split(word_matrix, y, test_size = 0.2, random_state = 42)

# use cross validation to choose C
c_param = {'C': [0.01, 0.1, 1, 10, 100]}
grid_search = GridSearchCV(SVC(kernel = "linear"), c_param, cv = 5)
grid_search.fit(X_train, y_train)
best_c = grid_search.best_params_['C']
print(f'Best C parameter: {best_c}')

# train final model with best c
linear_svm = SVC(kernel = "linear", C = best_c)
linear_svm.fit(X_train, y_train)

y_pred_linear = linear_svm.predict(X_valid)
print('Soft-Margin Linear SVM:')
print('Accuracy:', accuracy_score(y_valid, y_pred_linear))
print('Classification Report:')
print(classification_report(y_valid, y_pred_linear))

## Neural Network

Task 3: Investigate the impact of multiple hyperparameters. Compare the classification quality across four classification models in terms of F1 measure. Learn to manage overfitting or underfitting situations 

Task 4: Report your answers for each question and summarize your insights